<a href="https://colab.research.google.com/github/duyvm/leetcode-problems/blob/main/%5BMED%5D3342_Find_Minimum_Time_to_Reach_Last_Room_II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 3342. Find Minimum Time to Reach Last Room II

https://leetcode.com/problems/find-minimum-time-to-reach-last-room-ii/description/

There is a dungeon with `n x m` rooms arranged as a **grid**.

You are given a **2D array** `moveTime` of size `n x m`, where `moveTime[i][j]` represents the minimum time in seconds when you can start moving to that room.

You start from the room `(0, 0)` at time `t = 0` and *can move to an adjacent room*. Moving between **adjacent** rooms takes one second for one move and two seconds for the next, **alternating** between the two.

Return the **minimum time** to reach the room `(n - 1, m - 1)`.

Two rooms are adjacent if they share a common wall, either horizontally or vertically.

**Constraints:**
- `2 <= n == moveTime.length <= 750`
- `2 <= m == moveTime[i].length <= 750`
- `0 <= moveTime[i][j] <= 10`<sup>`9`</sup>

**Example 1**

Explanation: The minimum time required is `7` seconds.
  
  - At time `t == 4`, move from room `(0, 0)` to room `(1, 0)` in one second.
  
  - At time `t == 5`, move from room `(1, 0)` to room `(1, 1)` in two second.


In [2]:
test_case1 =  {
            "input": {
                "moveTime": [[0,4],[4,4]]
            },
            "output": 7
        }

**Example 2**

Explanation: The minimum time required is `6` seconds.

  - At time `t == 0`, move from room `(0, 0)` to room `(1, 0)` in one second.
  - At time `t == 1`, move from room `(1, 0)` to room `(1, 1)` in two second.
  - At time `t == 3`, move from room `(1, 1)` to room `(1, 2)` in one second.
  - At time `t == 4`, move from room `(1, 2)` to room `(1, 3)` in two second.

In [3]:
test_case2 =  {
            "input": {
                "moveTime": [[0,0,0,0],[0,0,0,0]]
            },
            "output": 6
        }

**Example 3**

In [4]:
test_case3 =  {
            "input": {
                "moveTime": [[0,1],[1,2]]
            },
            "output": 4
        }

In [5]:
test_cases = [test_case1, test_case2, test_case3]

In [6]:
def run_test_cases(solution, function_name):
    for i in range(len(test_cases)):
        run_test_case(solution, function_name, i)

def run_test_case(solution, function_name, i):
    test_case = test_cases[i]
    result = getattr(solution, function_name)(**test_case["input"])
    if result != test_case["output"]:
        print(f'Failed test case {i} with input: {test_case["input"]} and expected result: {test_case["output"]} vs actual result: {result}')

# Approach 1 - Dijkstra's Shortest Path Algorithm - Beat 36.05%

### Observations

- Use shortest path algorithm to solve this problem

- Used algorithm: Dijkstra algorithm to find shortest path from source to all others vertices

**Mapping between graph problem and this problem**

- Let 2D-array `minTime` with size `n x m` is the array stores minimum time moving from source `(0,0)` to vertex `(i,j)`

- Each cell is a vertex. We have total `n x m` vertices. Adjacent cells of cell `a` are neighbor vertices of vertex `a`. A vertex and its neighbor have an edge between them.

- The time moving between cells is the edge's distance. In this case, it is `1` or `2`. We must store the moving time that we use to move to source vertex.

- The minimum time can start moving to cell is the condition for moving to a vertex.

  - For example, moving from `a = (i,j)` to its neighbor vertex `b = (i+1,j)`.

  - We have `minTime[a] = x` and `moveTime[b] = y`

  - The `minTime[b]` is determine by `max(minTime[a], moveTime[b]) + movingTime`

- Each time, we consider the un-processed vertex with minimum time moving from source to it. We using min heap data structure for this task.

- Instead of pre-compute a vertext's neighbors, we can compute vertex's neighbors on the fly when process vertex. We use a delta array and apply it to current vertex to find its four neightbor vertices

- We do not store processed vertices. We can determine if it is processed or not by checking its current minTime with initialized value.

- Return early if we encounter (n-1, m-1) vertex

### Analysis

- Time complexity: `O(n * m * log(n * m))`


### Implementation

In [8]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    def minTimeToReach(self, moveTime: List[List[int]]) -> int:
        n = len(moveTime)
        m = len(moveTime[0])
        minTime = [[math.inf] * m for _ in range(n)]
        minTime[0][0] = 0

        movingTime = [[0] * m for _ in range(n)]
        movingTime[0][0] = 2

        minHeap = []

        # add source (minTime, i, j)
        heappush(minHeap, (0, 0, 0))

        neighborDirections = [(0,1), (0,-1), (1,0), (-1,0)]

        movingTimeAlternate = {1: 2, 2: 1}

        while minHeap:
            time, i, j = heappop(minHeap)

            # early return
            if i == n - 1 and j == m - 1:
                return time

            for delta_i, delta_j in neighborDirections:
                neighbor_i = i + delta_i
                neighbor_j = j + delta_j

                if 0 <= neighbor_i < n and 0 <= neighbor_j < m:
                    timeToNeighborVertex = max(time, moveTime[neighbor_i][neighbor_j]) + movingTimeAlternate[movingTime[i][j]]

                    updatedMinTime = min(timeToNeighborVertex, minTime[neighbor_i][neighbor_j])

                    if minTime[neighbor_i][neighbor_j] == math.inf:
                        heappush(minHeap, [timeToNeighborVertex, neighbor_i, neighbor_j])

                    minTime[neighbor_i][neighbor_j] = updatedMinTime

                    movingTime[neighbor_i][neighbor_j] = movingTimeAlternate[movingTime[i][j]]

In [9]:
run_test_cases(Solution(), "minTimeToReach")

# Approach 2 - Optimized approach 1 - Beat 42%

### Observations

- Although the time moving between cells is alternating between `1` and `2`. We do need to store this information.

- Instead we compute the moving time base on source vertex's coordinate `(i,j)`.Take a look at starting vertex `(0,0)`, we start moving with time `1`. For any neighbor vertices of `(0,0)`, the sum of its coordinate is odd by increasing by `1` (for example, (0,1) or (1,0)) and the next moving time from that neighbor is `2`. Next, the neighbor vertices will have sum of their coordinate is even and next moving time is `1`

- With this information, we can conclude that if sum of vertex's coordinate is even, the moving time from that vertex is `1` and vice versa.

- We can compute the moving time as follow:

        movingTime = (i+j) % 2 + 1


### Analysis

- Time complexity: `O(n * m * log(n * m))`

In [10]:
from re import L
from typing import List, Optional
from collections import defaultdict
from collections import deque
from heapq import heapify, heappush, heappop
from queue import PriorityQueue
from collections import Counter
import math

class Solution:

    def minTimeToReach(self, moveTime: List[List[int]]) -> int:
        n = len(moveTime)
        m = len(moveTime[0])
        minTime = [[math.inf] * m for _ in range(n)]
        minTime[0][0] = 0

        minHeap = []

        # add source (minTime, i, j)
        heappush(minHeap, (0, 0, 0))

        neighborDirections = [(0,1), (0,-1), (1,0), (-1,0)]

        while minHeap:
            time, i, j = heappop(minHeap)

            # early return
            if i == n - 1 and j == m - 1:
                return time

            for delta_i, delta_j in neighborDirections:
                neighbor_i = i + delta_i
                neighbor_j = j + delta_j

                if 0 <= neighbor_i < n and 0 <= neighbor_j < m:
                    timeToNeighborVertex = max(time, moveTime[neighbor_i][neighbor_j]) + (i+j)%2 + 1

                    updatedMinTime = min(timeToNeighborVertex, minTime[neighbor_i][neighbor_j])

                    if minTime[neighbor_i][neighbor_j] == math.inf:
                        heappush(minHeap, [timeToNeighborVertex, neighbor_i, neighbor_j])

                    minTime[neighbor_i][neighbor_j] = updatedMinTime

In [11]:
run_test_cases(Solution(), "minTimeToReach")